## Instalação Bibliotecas

In [ ]:
%pip install oracledb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 47.0 MB/s eta 0:00:00


In [ ]:
!pip install cx_Oracle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 8.3 MB/s eta 0:00:00


## Bibliotecas

In [ ]:
import oracledb
import cx_Oracle
import getpass
import pandas as pd
import random
from datetime import datetime, timedelta

- 1) Gere mais 50 produtos, 20 vendedores, 100 clientes, 1000 vendas.

In [ ]:
pw = getpass.getpass("Enter password: ")

connection = oracledb.connect(
    user="RM553808",
    password=pw,
    dsn="oracle.fiap.com.br:1521/orcl")

print("Successfully connected to Oracle Database")

cursor = connection.cursor()

Enter password: ··········
Successfully connected to Oracle Database


In [ ]:
# inserindo 50 produtos
for i in range(11, 62, 1):
    cursor.execute(f"""
        INSERT INTO produtos (produto_id, nome, preco)
        VALUES ({i}, 'Produto Eletrônico {i}', {random.uniform(100.0, 1000.0):.2f})
    """)

# inserindo 100 clientes
for i in range(31, 132, 1):
    cursor.execute(f"""
        INSERT INTO clientes (cliente_id, nome, email)
        VALUES ({i}, 'Cliente Genérico {i}', 'cliente{i}@exemplo.com')
    """)

# inserindo 20 vendedores
for i in range(11, 32, 1):
    cursor.execute(f"""
        INSERT INTO vendedores (vendedor_id, nome)
        VALUES ({i}, 'Vendedor {i}')
    """)

# inserindo 1000 vendas
for i in range(201, 1202, 1):
    data_venda = datetime.now() - timedelta(days=random.randint(1, 365))
    cursor.execute(f"""
        INSERT INTO vendas (venda_id, cliente_id, produto_id, vendedor_id, quantidade, data_venda)
        VALUES ({i}, {random.randint(31, 131)}, {random.randint(11, 61)}, {random.randint(11, 31)}, {random.randint(1, 5)}, TO_DATE('{data_venda.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'))
    """)

connection.commit()
print("Dados inseridos com sucesso.")

IntegrityError: ORA-00001: unique constraint (RM553808.SYS_C001627171) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/

- 2) Faça uma consulta  por Cliente, Produto, Vendedor e Data. Quando qualquer destes itens for branco (sem digitação) o programa deve resolver de duas formas em células difierentes. Uma solução é condicional dentro do próprio SQL e outra é resolvida no Python.

In [ ]:
cliente_nome = input("Digite o nome do cliente (ou deixe em branco): ")
produto_nome = input("Digite o nome do produto (ou deixe em branco): ")
vendedor_nome = input("Digite o nome do vendedor (ou deixe em branco): ")
data_venda = input("Digite a data da venda (ou deixe em branco): ")

try:
    # Consulta SQL com condições dinâmicas
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        WHERE (:cliente_nome IS NULL OR c.nome LIKE :cliente_nome)
        AND (:produto_nome IS NULL OR p.nome LIKE :produto_nome)
        AND (:vendedor_nome IS NULL OR vd.nome LIKE :vendedor_nome)
        AND (:data_venda IS NULL OR v.data_venda = TO_DATE(:data_venda, 'YYYY-MM-DD'))
        ORDER BY v.data_venda DESC
    """, {'cliente_nome': cliente_nome or None, 'produto_nome': produto_nome or None,
          'vendedor_nome': vendedor_nome or None, 'data_venda': data_venda or None})

    results = cursor.fetchall()

    if results:
        df = pd.DataFrame(results, columns=['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade', 'Valor Total Venda', 'Nome Vendedor', 'Data Venda'])
        print("Consulta com condições dinâmicas no SQL:")
        print(df)
    else:
        print("Nenhuma compra encontrada para os critérios informados.")

except cx_Oracle.DatabaseError as e:
    print("Erro ao buscar compras:", e)




Digite o nome do cliente (ou deixe em branco): 
Digite o nome do produto (ou deixe em branco): 
Digite o nome do vendedor (ou deixe em branco): 
Digite a data da venda (ou deixe em branco): 2024-03-09
Consulta com condições dinâmicas no SQL:
   ID Venda         Nome Cliente           Nome Produto   Preço  Quantidade  \
0       297  Cliente Genérico 49  Produto Eletrônico 59  218.36           2   
1       859  Cliente Genérico 68  Produto Eletrônico 52  392.61           3   

   Valor Total Venda Nome Vendedor Data Venda  
0             436.72   Vendedor 17 2024-03-09  
1            1177.83   Vendedor 28 2024-03-09  


- CONSULTA EM PYTHON

In [ ]:
try:
    cursor.execute("""
        SELECT v.venda_id, c.nome AS cliente_nome, p.nome AS produto_nome, p.preco, v.quantidade,
        (p.preco * v.quantidade) AS valor_total_venda, vd.nome AS vendedor_nome, v.data_venda
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.cliente_id
        JOIN produtos p ON v.produto_id = p.produto_id
        JOIN vendedores vd ON v.vendedor_id = vd.vendedor_id
        ORDER BY v.data_venda DESC
    """)

    results = cursor.fetchall()

    if results:
        df = pd.DataFrame(results, columns=['ID Venda', 'Nome Cliente', 'Nome Produto', 'Preço', 'Quantidade', 'Valor Total Venda', 'Nome Vendedor', 'Data Venda'])
        # Filtrando os resultados em Python conforme as entradas do usuário
        if cliente_nome:
            df = df[df['Nome Cliente'].str.contains(cliente_nome, case=False)]
        if produto_nome:
            df = df[df['Nome Produto'].str.contains(produto_nome, case=False)]
        if vendedor_nome:
            df = df[df['Nome Vendedor'].str.contains(vendedor_nome, case=False)]
        if data_venda:
            df = df[df['Data Venda'] == pd.to_datetime(data_venda)]

        if not df.empty:
            print("\nConsulta filtrada em Python:")
            print(df)
        else:
            print("\nNenhuma compra encontrada para os critérios informados.")
    else:
        print("\nNenhuma compra encontrada.")

except cx_Oracle.DatabaseError as e:
    print("\nErro ao buscar compras:", e)

finally:
    cursor.close()
    connection.close()


Consulta filtrada em Python:
   ID Venda         Nome Cliente           Nome Produto   Preço  Quantidade  \
0       859  Cliente Genérico 68  Produto Eletrônico 52  392.61           3   
1       297  Cliente Genérico 49  Produto Eletrônico 59  218.36           2   

   Valor Total Venda Nome Vendedor Data Venda  
0            1177.83   Vendedor 28 2024-03-09  
1             436.72   Vendedor 17 2024-03-09  


In [ ]:
df.head(5)


,ID Venda,Nome Cliente,Nome Produto,Preço,Quantidade,Valor Total Venda,Nome Vendedor,Data Venda
0,297,Cliente Genérico 49,Produto Eletrônico 59,218.36,2,436.72,Vendedor 17,2024-03-09
1,859,Cliente Genérico 68,Produto Eletrônico 52,392.61,3,1177.83,Vendedor 28,2024-03-09


- 3) Calcule a mediana por total de vendas por cliente via python mas selecionando por SQL

In [ ]:
cursor.execute("""
    SELECT c.nome AS cliente_nome, SUM(p.preco * v.quantidade) AS valor_total_venda
    FROM vendas v
    JOIN clientes c ON v.cliente_id = c.cliente_id
    JOIN produtos p ON v.produto_id = p.produto_id
    GROUP BY c.nome
""")
results = cursor.fetchall()

# Extraindo os valores do total de vendas para uma lista, pois o results nao é uma lista, apenas guarda o resultado do select
valor_total_vendas = [result[1] for result in results]

mediana = pd.Series(valor_total_vendas).median()
print(f"Mediana do total de vendas por cliente: {mediana:.2f}")


Mediana do total de vendas por cliente: 14426.05
